In [7]:
using Distributed

@everywhere using LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, Tables, TableOperations, StatsBase, FreqTables, DataFrames, FixedEffectModels

┌ Info: Precompiling FixedEffectModels [9d5cd8c9-2029-5cab-9928-427838db53e3]
└ @ Base loading.jl:1423
┌ Warning: Module StatsBase with build ID 14330325151201 is missing from the cache.
│ This may mean StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Warning: Module StatsBase with build ID 14330325151201 is missing from the cache.
│ This may mean StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1107
┌ Info: Skipping precompilation since __precompile__(false). Importing FixedEffectModels [9d5cd8c9-2029-5cab-9928-427838db53e3].
└ @ Base loading.jl:1124
┌ Info: Precompiling FixedEffects [c8885935-8500-56a7-9867-7708b20db0eb]
└ @ Base loading.jl:1423
┌ Warning: Module StatsBase with build ID 14330325151201 is missing from the cache.
│ This may mean StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91] does n

In [6]:
@everywhere mutable struct rlassoIV_arg
    x::DataFrame
    y::DataFrame
    d::DataFrame
    z::DataFrame
    select_Z::Bool
    select_X::Bool
    post::Bool
end

In [9]:
@everywhere function rlassoIVselectX(x, d, y, z, post = true)
    
    d = Matrix(self.d)
    z = Matrix(self.z)
    
    n = size(y, 1)
    numIV = size(z, 2)
    Z = hcat(x, z)
    lasso_d_x = rlasso(rlasso_arg(x, d, post))
    Dr = lasso_d_x["residuals"]
    lasso_y_x = rlasso(rlasso_arg(x, y, post))
    Yr = lasso_y_x["residuals"]
    Zr = zeros(n, numIV)
    for i in 1:numIV
        lasso_z_x = rlasso(rlasso_arg(x, z, post))
        Zr[:, i] = lasso_z_x["residuals"]
    end
    
    data = DataFrame(Yr = Yr, Dr = Dr, Zr = Zr)
    result = reg(data, @formula(Yr ~ (Dr ~ Zr)))
    se = FixedEffectModels.coeftable(result).cols[2]
    vcov = vcov(result)
    coef = coef(result)[1]
    res = Dict("coefficients" => coef, "vcov" => vcov, "se" => se)
    
    return res
end
    
    